In [ ]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.1 MB/s eta 0:00:00


In [ ]:
import requests
import gradio as gr
import base64
import io
import uuid
import time

# Inline config
class ConfigLLaVAVision:
    ENDPOINT_ID = "og65lbckc1lf24"
    API_KEY = "api key"
    BASE_URL = f"https://api.runpod.ai/v2/{ENDPOINT_ID}"


def image_to_base64(img):
    buffered = io.BytesIO()
    img.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode("utf-8")


def get_caption(image, prompt):
    if not image or not prompt:
        return "Please upload an image and enter a prompt."

    image_b64 = image_to_base64(image)

    payload = {
        "input": {
            "prompt": prompt,
            "source": image_b64
        }
    }

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {ConfigLLaVAVision.API_KEY}"
    }

    try:
        submit_response = requests.post(
            f"{ConfigLLaVAVision.BASE_URL}/run",
            json=payload,
            headers=headers
        )
        submit_response.raise_for_status()
        job_data = submit_response.json()
        job_id = job_data.get("id")

        if not job_id:
            return " Failed to submit job."

        # 2. Poll job status
        for attempt in range(30):  # Wait up to ~60 seconds
            time.sleep(2)
            status_response = requests.get(
                f"{ConfigLLaVAVision.BASE_URL}/status/{job_id}",
                headers=headers
            )
            status_response.raise_for_status()
            status_json = status_response.json()
            status = status_json.get("status")

            if status == "COMPLETED":
                output = status_json.get("output", {})
                print("Final Output:", output)
                if isinstance(output, dict):
                    return output.get("text", str(output))
                return str(output)

            elif status == "FAILED":
                return " Job failed during processing."

        return "⌛ Timed out waiting for response."

    except Exception as e:
        return f" Error: {str(e)}"

# Gradio Interface
gr.Interface(
    fn=get_caption,
    inputs=[
        gr.Image(type="pil", label="Upload Image"),
        gr.Textbox(
            lines=2,
            label="Prompt",
            value="Describe the image in detail like what are the objects and style and composition are included in simple one passage use simple English words.only one passage. "  # 📝 Embedded default
        )
    ],
    outputs="text",
    title="🦙 LLaVA Vision on Runpod (Async)",
    description="Submit an image + prompt and get a caption after ~30s from a Runpod serverless model."
).launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3ceb24fc58fb164706.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
